In [1]:
import itertools
from collections import OrderedDict
from allpairspy import AllPairs

In [2]:
class RFTS:
    def __init__(self, orderdict, orderdictALLPair, rules=None, rule2pair=None, default=None):
        self._allAttr = list(orderdict)
        self._param =[]
        self._param_space =[]
        for attr in self._allAttr:
            self._param.append(orderdict[attr])
        for attr in self._allAttr:
            self._param_space.append(orderdictALLPair[attr])            
        self._rules = rules
        self._default = default
        self._fifo=[]
        self._rule_2pair = rule2pair
    def printFifo(self):
        for i in self._fifo:
            print(i)
    def printFifoLen(self):
        print('FIFO Len: ', len(self._fifo))            
    def all_combination(self, fix_value=None, save=False):
        if fix_value !=None:
            new_param = self._fixParam(fix_value)
        else:
            new_param = self._param[:]
        listComb = list(itertools.product(*new_param))
        listComb = self._listToDict(listComb)
        if self._rules!=None:
            listComb = self._rules(listComb)
        print("All Comb Cases: ", len(listComb))
        if save == True:
            for tup in listComb:
                self._fifo.append(tup)
        return listComb
    def default_value_strategy(self, fix_value=None, save=False):
        if self._default == None:
            print("No Default Value")
            return
        _default = dict.copy(self._default)
        if fix_value !=None:
            new_param = self._fixParam(fix_value)
            for fix_attr, fix_choice in fix_value.items():
                if _default[fix_attr] not in fix_choice:
                    _default[fix_attr] = fix_choice[0]
        else:
            new_param = self._param[:]
        arr_dict=[]
        arr_dict.append(_default)
        for attr, param in zip(self._allAttr, new_param):
            for value in param:
                new_dict = dict.copy(_default) ##Copy Values
                if self._default[attr] != value:
                    new_dict[attr] = value
                    #print(new_dict)
                    arr_dict.append(new_dict)
        listComb = arr_dict
        if self._rules!=None:
            listComb = self._rules(listComb)   
        print("Default Cases: ", len(listComb))
        if save == True:
            for tup in listComb:
                self._fifo.append(tup)        
        return listComb
    def pairwise_comb(self, fix_value=None, save=False, n=2):
        if fix_value !=None:
            new_param = self._fixParam_space(fix_value)
        else:
            new_param = self._param_space[:]
        if self._rule_2pair != None:
            arr_dict=[pairs for _, pairs in enumerate(AllPairs(new_param, n=n, filter_func=self._rule_2pair))]
        else:
            arr_dict=[pairs for _, pairs in enumerate(AllPairs(new_param, n=n))]
        listComb = self._listToDict(arr_dict)
        print("PairWise Cases: ", len(listComb))
        if save == True:
            for tup in listComb:
                self._fifo.append(tup)        
        return listComb
    def addDefault(self, save=False, fix_value=None):
        _default = dict.copy(self._default)
        if fix_value !=None:
            for fix_attr, fix_choice in fix_value.items():
                if _default[fix_attr] not in fix_choice:
                    _default[fix_attr] = fix_choice[0]
        if save == True:
            self._fifo.append(_default)           
        return _default
    def printFifoResult(self):
        print("FIFO Size: ", len(self._fifo))
    def printFifoToFile(self):
        f = open("testData.txt", "w")
        for row in self._fifo:
            f.write(",")#price 
            f.write(",")#Mileage
            for cell in row.items():
                f.write(cell[1])
                f.write(",")
            f.write(",") #Valid
            f.write("\"\",") #Exp price
            f.write("\"\",") #Exp Period
            f.write("\"\"") #Exp Mileage
            f.write("\n")
        f.close()            
    def clearFIFO(self):
        self._fifo=[]
    def ruleSetFIFO(self):
        self._fifo = self._rules(self._fifo)
        return self._fifo
    def _listToDict(self, listComb):
        arr_dict=[]
        for i in range(len(listComb)):
            dict={}
            for attr, param in zip(self._allAttr, listComb[i]):
                dict[attr] = param
            arr_dict.append(dict)
        return arr_dict
    def _fixParam(self, fix_value):
        new_param = self._param[:]
        for i in fix_value.items():
            if i[0] in self._allAttr:
                index = self._allAttr.index(i[0])
                new_param[index] = i[1]
        return new_param
    def _fixParam_space(self, fix_value):
        new_param = self._param_space[:]
        for i in fix_value.items():
            if i[0] in self._allAttr:
                index = self._allAttr.index(i[0])
                new_param[index] = i[1]
        return new_param    

In [3]:
def get_unique(arr_dict):
    if(len(arr_dict)<=1):
        return arr_dict
    seen=[]
    for _dict in arr_dict:
        if _dict not in seen:
            seen.append(_dict)
    return seen

In [4]:
def rule(arr_dict):
    new_arr_dict=[]
    new_dict={}
    for _dict in arr_dict:
        new_dict = _dict
        #case NonDom
        if _dict['rangeType'] != "N":
            if _dict['seatType'] == "F":
                continue
            new_dict['lengthType'] = "_"
            new_dict['flightType'] = "_"
        #case econ
        if _dict['seatType'] != "E":
            new_dict['upgradeType'] = "_" 
        #case vip
        if _dict['passengerType'] != "V":
            new_dict['VIPType'] = "_"
        new_arr_dict.append(new_dict)
    new_arr_dict = get_unique(new_arr_dict)
    return new_arr_dict
def ruleForAllPair(row):
    n=len(row)
    if n>5:
        # case nonDom
        if  row[1]=="D" and (row[0] == 'F' or row[4] == "S" or row[4] == "L" or row[5] == "D" or row[5] == "I"):
            return False
        if  row[1]=="N" and (row[4] == "_" or row[5] == '_'):
            return False        
    if n>2:
        #case econ
        if row[0] != "E" and (row[2] == "U" or row[2] == "N"):
            return False
        if row[0] == "E" and (row[2] == "_"):
            return False        
    if n>7:
        #case vip
        if row[6] == "N" and (row[7] == "D" or row[7] == "G"):
            return False
        if row[6] == "V" and (row[7] == "_"):
            return False        
    return True

In [5]:
AIRTICKET = OrderedDict({
    "seatType": ["F", "B", "E"],
    "rangeType": ["D", "N"],
    "upgradeType": ["U", "N"],
    "tripType":["O", "R"],
    "lengthType":["S", "L"],
    "flightType":["D", "I"],
    "passengerType":["V", "N"],
    "VIPType":["D", "G"],
    "seasonType":["P", "N", "L"],
    "discountType":["N", "D", "J", "S"]
})
AIRTICKET_forALLPair = OrderedDict({
    "seatType": ["F", "B", "E"],
    "rangeType": ["D", "N"],
    "upgradeType": ["U", "N", '_'],
    "tripType":["O", "R"],
    "lengthType":["S", "L", '_'],
    "flightType":["D", "I", '_'],
    "passengerType":["V", "N"],
    "VIPType":["D", "G", '_'],
    "seasonType":["P", "N", "L"],
    "discountType":["N", "D", "J", "S"]
})
default={
    "seatType": "E",
    "rangeType": "N",
    "upgradeType": "N",
    "tripType": "R",
    "lengthType": "L",
    "flightType": "D",
    "passengerType": "N",
    "VIPType": "G",
    "seasonType": "N",
    "discountType": "D"
}

In [6]:
rfts=RFTS(AIRTICKET, AIRTICKET_forALLPair, rules=rule, rule2pair=ruleForAllPair, default=default)

First, Check the correctness of program as Safe keeper

In [7]:
print("legitimate Cases:")
rfts.all_combination()
fix_value = {"seatType": ["E"], "passengerType": ["V"], "rangeType": ["N"]}
print(fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["E"], "passengerType": ["V"], "rangeType": ["D"]}
print(fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["E"], "passengerType": ["N"], "rangeType": ["N"]}
print(fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["E"], "passengerType": ["N"], "rangeType": ["D"]}
print(fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["F", "B"], "passengerType": ["V"], "rangeType": ["N"]}
print(fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["F", "B"], "passengerType": ["V"], "rangeType": ["D"]}
print(fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["F", "B"], "passengerType": ["N"], "rangeType": ["N"]}
print(fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["F", "B"], "passengerType": ["N"], "rangeType": ["D"]}
print(fix_value)
_=rfts.all_combination(fix_value)

legitimate Cases:
All Comb Cases:  1368
{'seatType': ['E'], 'passengerType': ['V'], 'rangeType': ['N']}
All Comb Cases:  384
{'seatType': ['E'], 'passengerType': ['V'], 'rangeType': ['D']}
All Comb Cases:  96
{'seatType': ['E'], 'passengerType': ['N'], 'rangeType': ['N']}
All Comb Cases:  192
{'seatType': ['E'], 'passengerType': ['N'], 'rangeType': ['D']}
All Comb Cases:  48
{'seatType': ['F', 'B'], 'passengerType': ['V'], 'rangeType': ['N']}
All Comb Cases:  384
{'seatType': ['F', 'B'], 'passengerType': ['V'], 'rangeType': ['D']}
All Comb Cases:  48
{'seatType': ['F', 'B'], 'passengerType': ['N'], 'rangeType': ['N']}
All Comb Cases:  192
{'seatType': ['F', 'B'], 'passengerType': ['N'], 'rangeType': ['D']}
All Comb Cases:  24


Calculate the Invalid cases only:
1. Staff Discount in Peak Season for Econ without Upgrade
2. Staff Discount for Econ with Upgrade
3. First or Business has discount  

In [8]:
fix_value = {"seatType": ["E"], "upgradeType": ["N"], "seasonType": ["P"], "discountType": ["S", "J"]}
print('Case 1: ', fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["E"], "upgradeType": ["U"],  "discountType": ["S", "J"]}
print('Case 2: ', fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["F", "B"],  "discountType": ["S", "J", "D"]}
print('Case 3: ', fix_value)
_=rfts.all_combination(fix_value)

Case 1:  {'seatType': ['E'], 'upgradeType': ['N'], 'seasonType': ['P'], 'discountType': ['S', 'J']}
All Comb Cases:  60
Case 2:  {'seatType': ['E'], 'upgradeType': ['U'], 'discountType': ['S', 'J']}
All Comb Cases:  180
Case 3:  {'seatType': ['F', 'B'], 'discountType': ['S', 'J', 'D']}
All Comb Cases:  486


Calculate the Valid cases only:

In [9]:
fix_value = {"seatType": ["E"], "upgradeType": "N", "seasonType":["N", "L"]}
print('Case 1: ', fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["E"], "upgradeType": "N", "seasonType":["P"], "discountType": ["N", "D"]}
print('Case 2: ', fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["E"], "upgradeType": "U", "discountType": ["N", "D"]}
print('Case 3: ', fix_value)
rfts.all_combination(fix_value)
fix_value = {"seatType": ["F", "B"], "discountType": ["N"]}
print('Case 4: ', fix_value)
_=rfts.all_combination(fix_value)

Case 1:  {'seatType': ['E'], 'upgradeType': 'N', 'seasonType': ['N', 'L']}
All Comb Cases:  240
Case 2:  {'seatType': ['E'], 'upgradeType': 'N', 'seasonType': ['P'], 'discountType': ['N', 'D']}
All Comb Cases:  60
Case 3:  {'seatType': ['E'], 'upgradeType': 'U', 'discountType': ['N', 'D']}
All Comb Cases:  180
Case 4:  {'seatType': ['F', 'B'], 'discountType': ['N']}
All Comb Cases:  162


Total cases are 1368 (Invalid + valid) as expected

For Economy Class

In [10]:
rfts.clearFIFO()
fix_value = {"seatType": "E", "rangeType": "N", "upgradeType": "N", "tripType": "R", "lengthType": "L", "passengerType": "N", "discountType": ["N", "D"]}
print('Step E1: ', fix_value)
_=rfts.all_combination(fix_value, save=True)
fix_value = {"seatType": "E", "rangeType": "N", "upgradeType": "N", "seasonType":["N", "L"], "passengerType": "N"}# cover One-way, Indirect, Staff Discount
print('Step E2: ', fix_value)
_=rfts.pairwise_comb(fix_value=fix_value, save=True, n=2)
fix_value = {"seatType": "E", "rangeType": "N", "upgradeType": "N", "tripType": "R", "lengthType": "L",  "passengerType": "V", "seasonType":["N"], "discountType": ["N"]}# Cover Membership
print('Step E3: ', fix_value)
_=rfts.default_value_strategy(fix_value, save=True)
fix_value = {"seatType": "E",  "passengerType": "N", "discountType": ["N", "D"]}
print('Step E4: ', fix_value)
_=rfts.pairwise_comb(fix_value=fix_value, save=True, n=2)
rfts.printFifoLen()
rfts.ruleSetFIFO()
rfts.printFifoLen()
rfts.printFifoToFile()

Step E1:  {'seatType': 'E', 'rangeType': 'N', 'upgradeType': 'N', 'tripType': 'R', 'lengthType': 'L', 'passengerType': 'N', 'discountType': ['N', 'D']}
All Comb Cases:  12
Step E2:  {'seatType': 'E', 'rangeType': 'N', 'upgradeType': 'N', 'seasonType': ['N', 'L'], 'passengerType': 'N'}
PairWise Cases:  10
Step E3:  {'seatType': 'E', 'rangeType': 'N', 'upgradeType': 'N', 'tripType': 'R', 'lengthType': 'L', 'passengerType': 'V', 'seasonType': ['N'], 'discountType': ['N']}
Default Cases:  3
Step E4:  {'seatType': 'E', 'passengerType': 'N', 'discountType': ['N', 'D']}
PairWise Cases:  7
FIFO Len:  32
FIFO Len:  30


For Business Class

In [11]:
fix_value = {"seatType": ["B"], "rangeType": "N", "discountType": "N"}
print('Step B1: ', fix_value)
_=rfts.pairwise_comb(fix_value=fix_value, save=True, n=2)
fix_value = {"seatType": ["B"], "rangeType": "D", "discountType": "N"}
print('Step B2: ', fix_value)
_=rfts.default_value_strategy(fix_value=fix_value, save=True)
rfts.printFifoLen()
rfts.ruleSetFIFO()
rfts.printFifoLen()
rfts.printFifoToFile()

Step B1:  {'seatType': ['B'], 'rangeType': 'N', 'discountType': 'N'}
PairWise Cases:  7
Step B2:  {'seatType': ['B'], 'rangeType': 'D', 'discountType': 'N'}
Default Cases:  5
FIFO Len:  42
FIFO Len:  42


For First Class

In [12]:
fix_value = {"seatType": ["F"], "rangeType": "N", "tripType": "R", "lengthType": "L", "passengerType": "V", "discountType": ["N"]}
print('Step F1: ', fix_value)
_=rfts.pairwise_comb(fix_value=fix_value, save=True, n=2)
fix_value = {"seatType": ["F"], "passengerType": "N", "discountType": ["N"]}
print('Step F2: ', fix_value)
_=rfts.default_value_strategy(fix_value=fix_value, save=True)
rfts.printFifoLen()
rfts.ruleSetFIFO()
rfts.printFifoLen()
rfts.printFifoToFile()

Step F1:  {'seatType': ['F'], 'rangeType': 'N', 'tripType': 'R', 'lengthType': 'L', 'passengerType': 'V', 'discountType': ['N']}
PairWise Cases:  7
Step F2:  {'seatType': ['F'], 'passengerType': 'N', 'discountType': ['N']}
Default Cases:  6
FIFO Len:  55
FIFO Len:  55


InValid Cases

In [13]:
fix_value = {"seatType": ["E"], "upgradeType": ["N"], "seasonType": ["P"], "discountType": ["S"]}
rfts.addDefault(fix_value=fix_value, save=True)
fix_value = {"seatType": ["E"], "upgradeType": ["N"], "seasonType": ["P"], "discountType": ["J"]}
rfts.addDefault(fix_value=fix_value, save=True)
fix_value = {"seatType": ["E"], "upgradeType": ["U"], "discountType": ["S"]}
rfts.addDefault(fix_value=fix_value, save=True)
fix_value = {"seatType": ["E"], "upgradeType": ["U"], "discountType": ["J"]}
rfts.addDefault(fix_value=fix_value, save=True)
fix_value = {"seatType": ["B"],  "discountType": ["D"]}
print(fix_value)
_=rfts.pairwise_comb(fix_value=fix_value, save=True, n=2)
fix_value = {"seatType": ["F"], "passengerType": "V",  "discountType": ["D"]}
print(fix_value)
_=rfts.pairwise_comb(fix_value=fix_value, save=True, n=2)
fix_value = {"seatType": ["F"], "passengerType": "N",  "discountType": ["D"]}
_=rfts.addDefault(fix_value=fix_value, save=True)
rfts.printFifoLen()
rfts.ruleSetFIFO()
rfts.printFifoLen()
rfts.printFifoToFile()

{'seatType': ['B'], 'discountType': ['D']}
PairWise Cases:  8
{'seatType': ['F'], 'passengerType': 'V', 'discountType': ['D']}
PairWise Cases:  7
FIFO Len:  75
FIFO Len:  75


Additional space

In [14]:
fix_value = {"seatType": ["E"], "upgradeType": ["U"], "flightType":["D"]}
_=rfts.addDefault(fix_value=fix_value, save=True)
fix_value = {"seatType": ["B"], "discountType": ["J"]}
_=rfts.addDefault(fix_value=fix_value, save=True)
fix_value = {"seatType": ["F"], "discountType": ["S"]}
_=rfts.addDefault(fix_value=fix_value, save=True)

In [15]:
rfts.ruleSetFIFO()
fix_value = {"VIPType": ["_"]}
_=rfts.addDefault(fix_value=fix_value, save=True)
_=rfts.addDefault(fix_value=fix_value, save=True)
rfts.printFifoLen()
rfts.printFifo()
rfts.printFifoToFile()

FIFO Len:  80
{'seatType': 'E', 'rangeType': 'N', 'upgradeType': 'N', 'tripType': 'R', 'lengthType': 'L', 'flightType': 'D', 'passengerType': 'N', 'VIPType': '_', 'seasonType': 'P', 'discountType': 'N'}
{'seatType': 'E', 'rangeType': 'N', 'upgradeType': 'N', 'tripType': 'R', 'lengthType': 'L', 'flightType': 'D', 'passengerType': 'N', 'VIPType': '_', 'seasonType': 'P', 'discountType': 'D'}
{'seatType': 'E', 'rangeType': 'N', 'upgradeType': 'N', 'tripType': 'R', 'lengthType': 'L', 'flightType': 'D', 'passengerType': 'N', 'VIPType': '_', 'seasonType': 'N', 'discountType': 'N'}
{'seatType': 'E', 'rangeType': 'N', 'upgradeType': 'N', 'tripType': 'R', 'lengthType': 'L', 'flightType': 'D', 'passengerType': 'N', 'VIPType': '_', 'seasonType': 'N', 'discountType': 'D'}
{'seatType': 'E', 'rangeType': 'N', 'upgradeType': 'N', 'tripType': 'R', 'lengthType': 'L', 'flightType': 'D', 'passengerType': 'N', 'VIPType': '_', 'seasonType': 'L', 'discountType': 'N'}
{'seatType': 'E', 'rangeType': 'N', 'upgr